In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

#Data Source: https://drive.google.com/drive/folders/1tDkBFSXJz2bFcGR4AHW1nc8g1hLt_WVr?usp=sharing

Mounted at /content/gdrive


In [ ]:
TRAIN_PATH = "/content/gdrive/MyDrive/Neha/Train"
VAL_PATH = "/content/gdrive/MyDrive/Neha/Validate"

In [ ]:
import glob

In [ ]:
test = []
for i in glob.glob("/content/gdrive/MyDrive/Neha/Train/one/*"):
  test.append(i)

In [ ]:
len(test)

2816

In [ ]:
test1 = []
for i in glob.glob("/content/gdrive/MyDrive/Neha/Train/zero/*"):
  test1.append(i)

In [ ]:
len(test1)

3267

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2,activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 186624)            0

In [ ]:
from keras.preprocessing import image

# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/Neha/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical')

Found 6083 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'one': 0, 'zero': 1}

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    '/content/gdrive/MyDrive/Neha/Validate',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical' )

Found 1431 images belonging to 2 classes.


In [ ]:
tf.config.run_functions_eagerly(True)

hist = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs = 1,  
    validation_data = validation_generator,
    validation_steps=len(validation_generator)
)

model.save("/content/gdrive/MyDrive/Neha/Validate/dr_model_cnn.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


107/191 [===============>..............] - ETA: 53:18 - loss: 0.6962 - accuracy: 0.5348

In [ ]:


model = tf.keras.models.load_model("/content/gdrive/MyDrive/Neha/Validate/dr_model_cnn.h5")
import glob
test = []
for i in glob.glob("/content/gdrive/MyDrive/Neha/Validate/one/*"):
  testing = load_img(i, target_size=(224,224))
  testing
  x = img_to_array(testing)
  x = x/255
  x = np.expand_dims(x, axis=0)

  #model.predict_classes(x))
  predict_x=model.predict(x)
  classes_x=np.argmax(predict_x,axis=1)
  print(classes_x)
  print(model.predict(x))#confidence score

In [ ]:
import glob
test = []
for i in glob.glob("/content/gdrive/MyDrive/Neha/Validate/zero/*"):
  testing = load_img(i, target_size=(224,224))
  testing
  x = img_to_array(testing)
  x = x/255
  x = np.expand_dims(x, axis=0)
  predict_x=model.predict(x)
  classes_x=np.argmax(predict_x,axis=1)
  print(classes_x)
  #print(model.predict_classes(x))
  print(model.predict(x))#confidence score

In [ ]:
model

In [ ]:
import sklearn.metrics as metrics

Y_test = []
prediction_cnn = []

print('Confusion Matrix: CNN')
print(metrics.confusion_matrix(Y_test, prediction_cnn))
print('\nClassification Report:')
print(metrics.classification_report(Y_test, prediction_cnn))